Challenge Begins
---

As time flies, Now the time let us attend compettiion:

[Tbrain by Trend-Micro](https://tbrain.trendmicro.com.tw)  台灣 ETF 價格預測競賽


1. Huge data file: the way to read a data file, chunk by chunk,
```python
chunksize = 10 ** 6
for chunk in pd.read_csv(filename, chunksize=chunksize):
       process(chunk)
```    


    
|序號|欄位|說明|
|-:|-:|-:|
|0|ETFid|ETF 代號|
| 1| Mon_ud | 周一漲跌 (1:漲, 0:平, -1:跌) 50% |
| 2| Mon_cprice| 周一收盤價 (權重 50%) |
| 3| Tue_ud | 周二漲跌 (1:漲, 0:平, -1:跌) 50% |
| 4| Tue_cprice| 周二收盤價 (權重 50%) |
| 5| Wed_ud | 周三漲跌 (1:漲, 0:平, -1:跌) 50% |
| 6| Wed_cprice| 周三收盤價 (權重 50%) |
| 7| Thu_ud | 周四漲跌 (1:漲, 0:平, -1:跌) 50% |
| 8| Thu_cprice| 周四收盤價 (權重 50%) |
| 9| Fri_ud | 周五漲跌 (1:漲, 0:平, -1:跌) 50% |
| 10| Fri_cprice| 周五收盤價 (權重 50%) |

Sample Format

```
ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice
0050,1,87.15,1,88.05,-1,88.00,-1,87.25,1,87.45
...
```

In [ ]:
import numpy as np
from matplotlib import cm, pyplot as plt
import matplotlib.dates as dates
import pandas as pd
import datetime

from fbprophet import Prophet
from pandas_datareader import data as web
import datetime

import fix_yahoo_finance as yf
yf.pdr_override()


In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import time
today=time.strftime("%Y-%m-%d")
today

|code|ETF|code|ETF|
| ------| :------: | :------: |---|
|0050|元大台灣50|0058|富邦發達|
|0051|元大中型100 |0059|富邦金融|
|0052|富邦科技|006201|元大富櫃50|
|0053|元大電子|006203|元大MSCI台灣|
|0054|元大台商50|006204|永豐臺灣加權 |
|0055|元大MSCI金融|006208|富邦台50|
|0056|元大高股息|00690|兆豐藍籌30|
|0057|富邦摩台|00692|富邦摩台|
|00701|國泰臺灣低波動30|00713|元大台灣高息低波|

In [ ]:
# manually define the name of ETF's
etfs=['0050','0051','0052','0053','0054','0055','0056','0057','0058',
      '0059','006201','006203','006204','006208','00690','00692','00701','00702']

In [ ]:
import os, sys


Create a new directory if not exists,
```python
import os
if os.path.exists("data"):
   print("exist")
else:
   os.mkdir("data")
```    

In [ ]:
df0050_date=pd.read_csv("data/0050.csv",index_col='Date')

In [ ]:
df0050_date=df0050_date.rename(columns={"代碼": "code", "日期": "date",'中文簡稱':'Name',"開盤價(元)":'Open',
                            "最高價(元)":'High',"最低價(元)":'Low',"收盤價(元)":'Close',"total":"Volume"});


In [ ]:
df0050_date.head()

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# The coefficients
print('Prediction: \n', LinReg.intercept_,' + ',LinReg.coef_,'X_Close')
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y, Y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y, Y_pred))


FaceBook Prophet Introduction
---
One of most prediction package recently for time-series, especially for economical data; it acclaims:
1. simple, only 'ds' and 'y', date and observers quantity, are required;
- as usual usual, claim the model, training and predict. 

You should admire this for its compactness!

In [ ]:
from fbprophet import Prophet

In [ ]:
#dffb=pd.DataFrame({'ds':pd.to_datetime(df0050_date.index),'y':df0050_date['Close']})
dffb=pd.DataFrame()

In [ ]:
dffb['y'] = df0050_date['Close']
dffb['ds'] = pd.to_datetime(df0050_date.index)



In [ ]:
dffb.index=pd.to_datetime(dffb.index)

In [ ]:
dffb.head()

Modeling Prediction
---
```
Prophet(growth='linear', changepoints=None, n_changepoints=25, yearly_seasonality='auto',
        weekly_seasonality='auto', daily_seasonality='auto', holidays=None, 
        seasonality_prior_scale=10.0, holidays_prior_scale=10.0, 
        changepoint_prior_scale=0.05, mcmc_samples=0, interval_width=0.8, uncertainty_samples=1000)
```
- Input ```df.ds, df.y```: datetime and data we want to predict, define them if none;
- Output ```df.yhat, df.yhat_upper, df.yhat_lower```: 80% CI's

In [ ]:
s={'changepoint_prior_scale':  0.05, 'weekly_seasonality' : False,
    'daily_seasonality': False,'monthly_seasonality': True,
    'yearly_seasonality': True,'changepoints': None}

In [ ]:
#model = Prophet(mcmc_samples=500)
model = Prophet(daily_seasonality=s['daily_seasonality'],weekly_seasonality=s['weekly_seasonality'], 
                                  yearly_seasonality=s['yearly_seasonality'],
                                  changepoint_prior_scale=s['changepoint_prior_scale'],
                                  changepoints=s['changepoints'])
model.add_seasonality(name = 'monthly', period = 30.5, fourier_order = 5)

In [ ]:
model.fit(dffb);

In [ ]:
future = model.make_future_dataframe(periods=7)
future[-14:]

In [ ]:
forecast = model.predict(future)
forecast[['ds','yhat', 'yhat_lower', 'yhat_upper']].tail(14)

In [ ]:
model.plot(forecast)

In [ ]:
model.plot_components(forecast)
plt.show()

In [ ]:
dffb.y[-14:]

In [ ]:
obs=forecast[-90:]
model.plot(obs)

In [ ]:
dffb.y[-14:].plot()


In [ ]:
fb=forecast[['ds','yhat', 'yhat_lower', 'yhat_upper']][-21:-7]

In [ ]:
plt.figure(figsize=(12,6))
dffb.y[-14:].plot()
plt.plot(fb.ds,fb.yhat,label='pred')
plt.legend()
plt.ylim(80,85)

In [ ]:
df_pred=forecast[['ds','yhat', 'yhat_lower', 'yhat_upper']][-7:]

In [ ]:
df_pred['weekday']=df_pred.ds.dt.weekday_name

In [ ]:
df_pred['yhat']=round(df_pred.yhat,2)

In [ ]:
etfs=['0050','0051','0052','0053','0054','0055','0056','0057','0058',
      '0059','006201','006203','006204','006208','00690','00692','00701','00702']

In [ ]:
indexs=df_pred['ds'][1:-1]

In [ ]:
index=df_pred['ds'][1:-1]
columns = ['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice','Wed_ud','Wed_cprice',
           'Thu_ud','Thu_cprice','Fri_ud','Fri_cprice']

dfs= pd.DataFrame(columns=columns)
#dfs= pd.DataFrame(index=etfs, columns=columns)
# Fill zero's rater than Nan 
#dfs = dfs.fillna(0)

In [ ]:
dfs

In [ ]:
# 
df_pred['yesterday_close'] = df0050_date.Close[-1:].values[0]
#df_pred['yesterday_close'][0]=df0050_date.Close[-1:]
df_pred['yesterday_close'][1:] = df_pred['yhat'][:-1]
df_pred['price_change'] = df_pred['yhat'] - df_pred['yesterday_close']
df_pred['sign'] = np.sign(df_pred['price_change']).astype('int')

In [ ]:
df_pred

In [ ]:
etfs0s=['0050']

In [ ]:
for etfs0 in etfs0s:
    a=df_pred.sign[1:-1].values
    b=df_pred.yhat[1:-1].values
    record=[etfs0]
    for i in range(len(a)):
        record.append(a[i])
        record.append(b[i])
    print(record)    

In [ ]:
# insert the final result in defaulted format
dfs.loc[0]=record

In [ ]:
dfs

Last Note
---
- Here, we shows how to create the result for id=`0050`. You can create the function in which load all the data, create fbprophet model, fit, etc. After completing work, write all the data into dfs as asked.
- Next week, append the last true data into old df and re-do the above work.
- <font color="red">Check the result with the exact data after while</font>.